In [ ]:
import numpy as np
import random
import os
import shutil
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
from re import X
import json
from PIL import Image
import csv
import cv2

In [ ]:
shutil.rmtree("./yolov5", ignore_errors=True)
shutil.rmtree("./yolov5x.pt", ignore_errors=True)

In [ ]:
SEED = 1988

random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
SEED

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
Path("./yolov5/yolov5/data/images/train").mkdir(parents=True, exist_ok=True)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
filess = os.listdir('/content/gdrive/MyDrive/CarPlate/train')

In [ ]:
for i in filess:
    shutil.copy2('/content/gdrive/MyDrive/CarPlate/train/' + i, './yolov5/yolov5/data/images/train')

In [ ]:
train = pd.read_csv("/content/gdrive/MyDrive/CarPlate/train.csv")

In [ ]:
cars = {'car': 1 }

In [ ]:
from collections import defaultdict
defdict = defaultdict(lambda: [])
for i in range(len(train)):
    temp = train.iloc[i]
    x1 = abs(temp['x1']) 
    x3 = (temp['x3']) 
    y1 = (temp['y1'])
    y3 = (temp['y3'])
    w = abs(x3 - x1)
    h = abs(y3 - y1)
    defdict[temp['file']].append([(x1 + w / 2)/temp['width'], (y1 + h / 2)/temp['height'], w/temp['width'], h/temp['height'], 'car'])

In [ ]:
from pathlib import Path
def create_dataset(dictr):
    labels_path = Path("./yolov5/yolov5/data/labels/train/")
    labels_path.mkdir(parents=True, exist_ok=True)
    for img_id, row in enumerate(tqdm(dictr)):
        image_name = row
        label_name = f"{image_name[:-4]}.txt"
        with (labels_path / label_name).open(mode="w") as label_file:
            for a in dictr[row]:
                category_idx = cars[a[4]] - 1
                label_file.write(
                    f"{category_idx} {a[0]} {a[1]} {a[2]} {a[3]}\n"
                )

In [ ]:
create_dataset(defdict)

In [ ]:
f = open('yolov5/data/data1.yaml', 'w+')
f.write('train: ./yolov5/data/images/train' + '\n')
f.write('val: ./yolov5/data/images/train' + '\n')
f.write('nc: 1' + '\n')
f.write("names: ['car']" + '\n')
f.close()

In [ ]:
%pip install -q wandb
import wandb

In [ ]:
os.environ['WANDB_MODE']="dryrun"

In [ ]:
!python ./yolov5/train.py --img 1024 --batch 6 --epochs 20   \
  --data ./yolov5/data/data1.yaml --cfg ./yolov5/models/yolov5x.yaml --weights yolov5x.pt \
  --name yolov5x_plates  --exist-ok

In [ ]:
!python ./yolov5/detect.py --weights /content/yolov5/runs/train/yolov5x_plates/weights/best.pt \
  --source /content/gdrive/MyDrive/CarPlate/test --save-txt \
  --name exp2 --exist-ok

In [ ]:
def IOU(box1, box2):
  x1, y1, x2, y2 = box1
  x3, y3, x4, y4 = box2
  x_inter1 = max(x1, x3)
  y_inter1 = max(y1, y3)
  x_inter2 = min(x2, x4)
  y_inter2 = min(y2, y4)
  width_inter = abs(x_inter2 - x_inter1)
  height_inter = abs(y_inter2 - y_inter1)
  area_inter = width_inter * height_inter
  width_box1 = abs(x2 - x1)
  height_box1 = abs(y2 - y1)
  width_box2 = abs(x4 - x3)
  height_box2 = abs(y4 - y3)
  area_box1 = width_box1 * height_box1
  area_box2 = width_box2 * height_box2
  area_union = area_box1 + area_box2 - area_inter
  iou = area_inter / area_union
  return iou


In [ ]:
filess = os.listdir('/content/yolov5/runs/detect/exp2/labels')
sum = 0
plus = 0

header = ['file', 'x1', 'y1', 'x3', 'y3', 'text']
rows = []

for item in filess:
  if '.txt' in item:
    imgPath = item.replace('.txt', '.jpg')
    filepath = '/content/gdrive/MyDrive/CarPlate/test/'+ imgPath
    img = Image.open(filepath)
    width = img.width
    height = img.height

    res = pd.read_csv('/content/yolov5/runs/detect/exp2/labels/' + item, sep=" ", header = None)

    xC =  res[1][0] 
    wO =  res[3][0] 

    yC =  res[2][0] 
    hO =  res[4][0] 

    x1 = xC * width - (wO * width) / 2
    y1 = yC * height - (hO * height) / 2

    x3 = x1 + wO * width
    y3 = y1 + hO * height

    row = train[train['file'] == imgPath]
    Ax1 = row.x1.item()
    Ay1 = row.y1.item()
    Ax3 = row.x3.item()
    Ay3 = row.y3.item()

    rows.append([imgPath, x1, y1, x3, y3, train[train['file'] == imgPath].text.item()])
    sum = sum + 1
    result = IOU([Ax1, Ay1, Ax3, Ay3], [x1, y1, x3, y3])

    if result > 0.5:
      plus = plus + 1

print('Accuracy: ', plus / sum)

In [ ]:
with open('/content/test.csv', 'w', encoding='UTF8', newline='') as f:
     writer = csv.writer(f)

     writer.writerow(header)

     writer.writerows(rows)

In [ ]:
test = pd.read_csv("/content/test.csv")

In [ ]:
!pip install easyocr

In [ ]:
import easyocr

header = ['text','easyOCR']
rows = []

for index, row in test.iterrows():
    filePath = row.file
    
    img = cv2.imread('/content/gdrive/MyDrive/CarPlate/test/' + filePath)
    
    x1 = round(row.x1)
    y1 = round(row.y1)
    x3 = round(row.x3)
    y3 = round(row.y3)

    cropped_image = img[y1:y3, x1:x3]

    reader = easyocr.Reader(['ru'])
    result = reader.readtext(cropped_image)

    easyOCR = ''

    if len(result) > 0:
        easyOCR = result[0][1]
    rows.append([row.text, easyOCR])

In [ ]:
!pip install pybind11
!pip install fastwer

In [ ]:
import fastwer

header = ['output', 'ref', 'cer']
numberList = rows
resultList = []
for i in numberList:
  ref = i[0].upper()
  output = i[1].upper()
  for ch in [' ', '|', '/', ']', '[', '%', '-', '"', "'", '(', '_', ':', '.', '#', '{', '₽', '*', '>', '}', '+']:
     output = output.replace(ch,'')
  cer = fastwer.score_sent(output, ref, char_level=True)
  resultList.append([output, ref, cer])

In [ ]:
with open('/content/cer.csv', 'w', encoding='UTF8', newline='') as f:
     writer = csv.writer(f)

     writer.writerow(header)

     writer.writerows(resultList)

In [ ]:
cer = pd.read_csv('/content/cer.csv')
cer

In [ ]:
print(f'Mean CER = {cer.cer.mean()}%')